# Feature Extraction

Since the filters have been explored, the data displayed in spectrograms can now be written to `csv`-files for later training of the NN. The amount of data per soundfile is calculated with the following formula:

```
size_in_b = n_frames * n_bands * sizeof(float32)
```

This is one of the major strenghts of this method: instead of storing `512` bins `1024` times per second, the octave bank only creates `31` bins `24` times a second. 

In [14]:
import os
from octafilt3r import filter as o3f
from octafilt3r import weightings as o3w
import librosa as lr
import numpy as np
import pipeline.config as conf

In [15]:
param_dict = conf.open_params()

In [16]:
# def spec(y, sr):
#     X = lr.stft(y)
#     Xdb = lr.power_to_db(abs(X)**2, ref = 1)
#     plt.figure(figsize=(14, 5))
#     lr.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='log', cmap = 'rainbow')
#     plt.colorbar()    

# y, sr = lr.load("demo_data/whitenoise.wav", sr=param_dict["fs"], mono=True, duration=param_dict["max_dur"])
# spec(y, sr)
# y_a = A_weight(y, sr)
# spec(y_a, sr)

In [17]:
files_ignored = 0
tot_cnt = 0

df = []
labels = []
leq = []
for class_ in os.listdir(param_dict["source_dir"]):
    cnt = 1
    for filename in os.listdir(f'{param_dict["source_dir"]}{class_}'):

        print(f'Currently extracting features in "{class_}" from {filename} (# {cnt})')

        wav = f'{param_dict["source_dir"]}{class_}/{filename}'
        y, sr = lr.load(wav, sr=param_dict["fs"], mono=True, duration=param_dict["max_dur"])

        y = o3w.A_weight(y, sr)

        wins = int(len(y)/param_dict["window_size"])
        if wins == 0:
            files_ignored += 1
            print('*ignored, file under 1 second long*')
            continue    # skip files which are shorter than 1s

        # obtain features of whole file
        feats, fcs, lq = o3f.rolling_oct_bank(y, 
                                        param_dict["fs"], 
                                        ratio=param_dict["oct_bw_ratio"], 
                                        fmax=param_dict["fmax"], 
                                        fmin=param_dict["fmin"], 
                                        frame_size=param_dict["frame_size"], 
                                        n_decimations=param_dict["dec_stages"])

        print(f'\tSplitting "{filename}" into {wins} 1-second part(s)')
        for win in range(wins):    # ignore parts of file which is not in a full window
            cur_feats = feats[param_dict["fps"] * win:param_dict["fps"] * (win + 1)]
            df.append(cur_feats)
            labels.append(class_)
            leq.append(lq[win])
        cnt += 1

    tot_cnt += cnt


df = np.dstack(df)
df = np.transpose(df, (2, 0, 1))

df_2d = df.reshape(df.shape[0], -1)
np.savetxt(param_dict["extr_txt_fp"], df_2d)
with open(param_dict["label_txt_fp"], "w") as f:
    for i in labels:
        f.writelines(i + '\n')
with open(param_dict["leq_txt_fp"], "w") as f:
    for i in leq:
        f.writelines(str(i) + '\n')        


print(f'----- done. -----\nTotal file count: {tot_cnt}\nFiles ignored: {files_ignored}\nExtracted {len(df)} windows from {tot_cnt-files_ignored} Files')
print(f'Shape of dataframe: {df.shape}')
print(f'Shape of labels: {np.shape(labels)}')
print(f'Shape of LEQs: {np.shape(leq)}')

Currently extracting features in "multi" from audiocheck.net_100Hz_-5dBFS_1000Hz_-17dBFS_10s.wav (# 1)
	Splitting "audiocheck.net_100Hz_-5dBFS_1000Hz_-17dBFS_10s.wav" into 10 1-second part(s)
Currently extracting features in "multi" from audiocheck.net_600Hz_-5dBFS_1200Hz_-10dBFS_10s.wav (# 2)
	Splitting "audiocheck.net_600Hz_-5dBFS_1200Hz_-10dBFS_10s.wav" into 10 1-second part(s)
Currently extracting features in "multi" from audiocheck.net_600Hz_-5dBFS_900Hz_-10dBFS_10s.wav (# 3)
	Splitting "audiocheck.net_600Hz_-5dBFS_900Hz_-10dBFS_10s.wav" into 10 1-second part(s)
Currently extracting features in "multi" from fade1_audiocheck.net_100Hz_-5dBFS_1000Hz_-17dBFS_10s.wav (# 4)
	Splitting "fade1_audiocheck.net_100Hz_-5dBFS_1000Hz_-17dBFS_10s.wav" into 10 1-second part(s)
Currently extracting features in "multi" from fade1_audiocheck.net_600Hz_-5dBFS_1200Hz_-10dBFS_10s.wav (# 5)
	Splitting "fade1_audiocheck.net_600Hz_-5dBFS_1200Hz_-10dBFS_10s.wav" into 10 1-second part(s)
Currently extracti